In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, LassoLarsCV
import warnings
import sys
import os
import pickle
# ignoramos os resultados NaN das funções pois vamnos zera-los
np.seterr(invalid='ignore')
# nao quero warning de convergência
warnings.filterwarnings('ignore')

In [2]:
def getResults(fname):
    dataset = []
    algoritmo = []
    msre_l = []
    if os.path.exists(fname):
        fw = open(fname, 'rb')
        dataset, algoritmo, msre_l  = pickle.load(fw)
        fw.close()
    return dataset, algoritmo, msre_l

In [3]:
def storeResults(dataset, algoritmo, msre_l, fname):
    fw = open(fname, 'wb')
    pickle.dump((dataset, algoritmo, msre_l), fw)
    fw.close()

In [4]:
def msre(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
def importaDados(fname):
    dataset = np.loadtxt(fname, delimiter=",")
    X = dataset[:, :-1]
    y = dataset[:, -1]
    return (X, y)

In [32]:
def geraRede(X, n_inter, inter_min=0, inter_max=3):
    n_inputs = X.shape[1]
    rede = np.random.randint(inter_min, inter_max, size=(n_inputs, n_inter))
    for i in range(0, n_inputs):
        if 0 in X[:,i]:
            rede[i,:] = np.absolute(rede[i,:])
    return rede

In [33]:
def transformData(X, rede):
    n_rows = X.shape[0]
    n_inter = rede.shape[1]
    layers = np.ndarray((n_rows, 3*n_inter))
    for i in range(0, n_inter*3, 3):
        power = X**rede[:,int(i/3)]
        layers[:,i] = np.prod(power,axis=1) # id
        layers[:,i+1] = np.cos(layers[:,i]) # cos
        layers[:,i+2] = np.sqrt(layers[:,i]) # sqrt
    cols = np.any(np.isnan(layers), axis=0)
    layers[:, cols] = 0
    cols = np.any(np.isinf(layers), axis=0)
    layers[:, cols] = 0
    return layers

In [34]:
def fit(X, y, algoritmo, n_inter, inter_min=0, inter_max=3):
    n_inputs = X.shape[1]
    rede = geraRede(X, n_inter, inter_min, inter_max)
    X_transf = transformData(X, rede)
    if algoritmo == 'lasso':
        modelo = LassoCV(max_iter=5e4, cv=3)
    elif algoritmo == 'lassoLars':
        modelo = LassoLarsCV(max_iter=5e4, cv=3)
    modelo.fit(X_transf, y)
    return rede, modelo

In [9]:
def predict(X, rede, modelo):
    X_transf = transformData(X, rede)
    y_hat = modelo.predict(X_transf)
    return y_hat

In [62]:
def expression(rede, coef):
    n_inter = rede.shape[1]
    n_inputs = rede.shape[0]
    fun = ['', 'cos', 'sqrt']
    expr = ''
    for i in range(n_inter):
        inter = ''
        for j in range(n_inputs):
            if rede[j,i] != 0:
                inter = inter + f'x{j}^{rede[j,i]}*'
        inter = inter[:-1]
        transf = ''
        for f in range(3):
            if coef[i*3+f] != 0:
                transf = transf + f'{coef[i*3+f]:.3}*{fun[f]}({inter}) + '
        expr = expr + transf
    expr = expr[:-2]
    return expr

In [66]:
# faça esse somente para rodar todos os testes novamente
dataset_l = []
algoritmo_l = []
msre_l = []

datasets = ['airfoil', 'concrete', 'cpu', 'energyCooling', 'energyHeating', 'forestfires', 'towerData', 'wineRed', 'wineWhite', 'yacht']
algoritmos = ['lassoLars', 'lassoLars', 'lassoLars', 'lassoLars', 'lassoLars', 'lasso', 'lassoLars', 'lassoLars', 'lassoLars', 'lassoLars']
ninters = [10, 500, 1000, 500, 500, 10, 10, 100, 100, 100]
inter_mins = [-2, -2, -3, -2, 0, -2, 0, -1, -1, -3]
inter_maxs = [2, 2, 3, 2, 4, 2, 2, 1, 1, 3]
pastas = ['0', '1', '2', '3', '4']

for i in range(10):
    dataset_l.append(datasets[i])
    algoritmo_l.append(f'{algoritmos[i]} {ninters[i]} ({inter_mins[i]} {inter_maxs[i]})')
    msre_total = 0
    coefs_l = []
    exprs_l  =[]
    print(f'dataset {datasets[i]}')
    for pasta in pastas:
        fileTrain = 'datasets/' + datasets[i] + '-train-' + pasta + '.dat'
        fileTest = 'datasets/' + datasets[i] + '-test-' + pasta + '.dat'
        X_train, y_train = importaDados(fileTrain)
        X_test, y_test = importaDados(fileTest)
        n = X_train.shape[1]
        rede, modelo = fit(X_train, y_train, algoritmos[i], ninters[i]*n, inter_mins[i], inter_maxs[i]+1)
        y_hat = predict(X_test, rede, modelo)
        msre_total += msre(y_test, y_hat)
        coef = modelo.coef_
        coefs_l.append(np.count_nonzero(coef))
        exprs_l.append(expression(rede, coef))
    m = len(pastas)
    msre_medio = msre_total/m
    msre_l.append(msre_medio)
    min_i = np.argmin(coefs_l)
    print('total de coeficientes não-zeros:', coefs_l[min_i])
    print('expressão:')
    print(exprs_l[min_i])
    with open('tests.txt', 'a') as myfile:
        myfile.write(f'{datasets[i]},{algoritmos[i]} {ninters[i]} ({inter_mins[i]} {inter_maxs[i]}),{msre_medio}\n')
    print(f'msre: {datasets[i]},{algoritmos[i]} {ninters[i]} ({inter_mins[i]} {inter_maxs[i]}),{msre_medio}')

storeResults(dataset_l, algoritmo_l, msre_l, 'tests.pkl')

dataset airfoil
total de coeficientes não-zeros: 38
expressão:
2.81e-06*(x0^2*x1^1*x2^2*x4^1) + -0.0412*sqrt(x0^2*x1^1*x2^2*x4^1) + -5.1e-21*(x0^2*x2^-2*x3^1*x4^-2) + 0.0201*cos(x0^2*x2^-2*x3^1*x4^-2) + -0.291*cos(x0^2*x1^1*x2^2*x3^1*x4^-1) + 2.07*cos(x1^2*x4^2) + -6.72e-17*(x0^2*x1^2*x2^-1*x3^1*x4^-1) + 0.111*cos(x0^1*x2^1*x3^-2*x4^-2) + -0.0188*sqrt(x0^1*x2^1*x3^-2*x4^-2) + 0.129*(x1^1*x2^2*x3^1) + -0.000311*cos(x1^1*x2^2*x3^1) + 2.3e+05*cos(x1^1*x2^1*x4^2) + 8.02e-06*(x3^-1*x4^-2) + 0.0358*sqrt(x3^-1*x4^-2) + -0.2*(x0^-1*x2^-1*x4^-1) + -2.03*cos(x0^-1*x2^-1*x4^-1) + 3.96e+11*cos(x0^-1*x1^2*x2^1*x3^-2*x4^1) + -1.71e+02*(x0^-2*x1^1*x3^-2*x4^-2) + -1.65e+07*(x0^-2*x2^2*x3^-2*x4^-1) + -2.84e+03*sqrt(x0^-2*x2^2*x3^-2*x4^-1) + -3.68*sqrt(x0^1*x1^2*x3^-2*x4^2) + -0.0816*cos(x1^1*x2^-1*x3^2) + 0.00116*sqrt(x1^1*x2^-1*x3^2) + 5.55*sqrt(x0^-1*x1^1) + -2.02*(x0^-1*x1^1*x2^-1*x3^1*x4^2) + 22.0*cos(x0^-1*x1^1*x2^-1*x3^1*x4^2) + 0.0181*(x2^1*x3^-2*x4^-2) + 2.69e+03*sqrt(x0^-2*x2^2*x3^2*x4^2) + 1.

total de coeficientes não-zeros: 30
expressão:
-0.189*(x0^-1*x1^1*x4^-1*x7^-1*x8^1*x9^-1*x10^-1) + -0.123*(x0^1*x1^1*x7^-1*x10^-1) + -0.162*(x1^1*x3^-1*x4^1*x5^1*x6^-1*x7^1*x9^-1) + 46.8*cos(x1^1*x3^-1*x4^1*x5^1*x6^-1*x7^1*x9^-1) + -0.0193*(x0^-1*x3^-1*x4^1*x5^1*x6^1*x7^-1*x8^1*x9^1*x10^-1) + 0.194*cos(x4^-1*x7^-1*x9^-1*x10^-1) + 0.112*sqrt(x4^-1*x8^-1*x10^1) + -0.0713*cos(x0^-1*x1^-1*x2^1*x4^-1*x5^1*x6^-1*x7^-1*x8^-1*x9^1*x10^1) + 3.1*cos(x0^-1*x1^-1*x2^1*x3^1*x4^-1*x5^-1*x7^-1*x8^-1*x9^-1*x10^-1) + -0.00459*cos(x0^1*x1^1*x3^1*x4^-1*x7^-1*x8^-1*x10^-1) + -0.0507*cos(x1^1*x4^-1*x6^-1*x7^1*x10^1) + 0.00654*cos(x1^-1*x2^1*x3^1*x4^1*x5^1*x7^-1*x8^1*x9^1*x10^-1) + 0.00237*cos(x1^-1*x4^-1*x5^1*x6^1*x8^-1*x9^1*x10^-1) + 0.397*cos(x0^-1*x1^1*x3^1*x4^-1*x6^-1*x7^-1*x9^-1) + 0.0255*cos(x0^-1*x1^-1*x3^-1*x6^1*x7^1*x9^1*x10^-1) + 0.714*(x0^-1*x8^-1*x10^1) + -0.921*cos(x0^-1*x8^-1*x10^1) + -0.00486*(x0^-1*x2^1*x6^1*x8^1*x9^1*x10^-1) + -75.2*(x0^-1*x1^1*x3^-1*x6^-1*x7^-1*x8^1*x9^-1*x10^-1) + 0.0001

In [73]:
# forestfires
i = 5
msre_total = 0
print(f'dataset {datasets[i]}')
for pasta in pastas:
    print(f'pasta {pasta}')
    fileTrain = 'datasets/' + datasets[i] + '-train-' + pasta + '.dat'
    fileTest = 'datasets/' + datasets[i] + '-test-' + pasta + '.dat'
    X_train, y_train = importaDados(fileTrain)
    X_test, y_test = importaDados(fileTest)
    n = X_train.shape[1]
    rede, modelo = fit(X_train, y_train, algoritmos[i], ninters[i]*n, inter_mins[i], inter_maxs[i]+1)
    y_hat = predict(X_test, rede, modelo)
    msre_total += msre(y_test, y_hat)
    coef = modelo.coef_
    print('total de coeficientes não-zeros:', np.count_nonzero(coef))
    print('expressão:')
    print(expression(rede, coef))
m = len(pastas)
msre_medio = msre_total/m
print(f'msre: {datasets[i]},{algoritmos[i]} {ninters[i]} ({inter_mins[i]} {inter_maxs[i]}),{msre_medio}')

dataset forestfires
pasta 0
total de coeficientes não-zeros: 1
expressão:
-2.29e-21*(x0^2*x1^2*x2^1*x3^2*x4^2*x5^2*x6^1*x7^1*x8^1) 
pasta 1
total de coeficientes não-zeros: 0
expressão:

pasta 2
total de coeficientes não-zeros: 1
expressão:
6.73e-34*(x1^2*x2^2*x3^2*x4^2*x7^1*x8^2) 
pasta 3
total de coeficientes não-zeros: 1
expressão:
2.28e-33*(x0^2*x2^1*x3^2*x4^2*x5^1*x6^1*x7^1*x8^1) 
pasta 4
total de coeficientes não-zeros: 0
expressão:

msre: forestfires,lasso 10 (-2 2),52.00171410752951


In [53]:
# faça esse ao invés da acima para carregar os resultados salvos
dataset_l, algoritmo_l, msre_l = getResults('tests.pkl')

In [67]:
# gerar o dataframe de resultados
df_IT = pd.DataFrame({'dataset':dataset_l, 'algoritmo':algoritmo_l, 'rmse':msre_l})

In [68]:
# salvar o df no arquivo
fw = open('df_IT.pkl', 'wb')
pickle.dump(df_IT, fw)
fw.close()

In [10]:
# carregar o df do arquivo de resultados
fw = open('df_IT.pkl', 'rb')
df_IT = pickle.load(fw)
fw.close()

In [69]:
# carregar o df do arquivo de resultados dos outros algoritmos
fw = open('df_GS.pkl', 'rb')
df_GS = pickle.load(fw)
fw.close()

In [70]:
# unir os resultados desses testes com os testes do GridSearch
df = pd.concat([df_IT, df_GS], ignore_index=True)

In [71]:
datasets = df_IT.dataset.unique()
for D in datasets:
    print('\n', D)
    print(df.loc[df['dataset'] == D])


 airfoil
              algoritmo  dataset      rmse
0   lassoLars 10 (-2 2)  airfoil  2.837423
28                Ridge  airfoil  4.862221
29                Lasso  airfoil  6.142515
30                 Lars  airfoil  4.819300
31           ElasticNet  airfoil  6.169792
32              XGBoost  airfoil  2.186636
33                  MLP  airfoil  4.811583

 concrete
               algoritmo   dataset       rmse
1   lassoLars 500 (-2 2)  concrete  32.790966
46                 Ridge  concrete  10.444737
47                 Lasso  concrete  10.446343
48                  Lars  concrete  10.443756
49            ElasticNet  concrete  10.446860
50               XGBoost  concrete   4.750018
51                   MLP  concrete   7.709026

 cpu
                algoritmo dataset       rmse
2   lassoLars 1000 (-3 3)     cpu  28.922440
22                  Ridge     cpu  34.904959
23                  Lasso     cpu  42.059112
24                   Lars     cpu  34.546549
25             ElasticNet     cpu  4